<a href="https://colab.research.google.com/github/sudarshan-koirala/youtube-stuffs/blob/main/langchain/AutoGPT_with_LangChain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AutoGPT with LangChain
- https://python.langchain.com/en/latest/use_cases/autonomous_agents/autogpt.html
- https://github.com/Significant-Gravitas/Auto-GPT

## Install necessary packages

In [22]:
%%capture
!pip install langchain google-search-results faiss-cpu openai tiktoken

## Setup Tools

In [48]:
import os
os.environ['SERPAPI_API_KEY'] = "SERPAPI_API_KEY"

In [45]:
os.environ['OPENAI_API_KEY'] = "OPENAI_API_KEY"

In [52]:
from langchain.utilities import SerpAPIWrapper
from langchain.agents import Tool
from langchain.tools.file_management.write import WriteFileTool
from langchain.tools.file_management.read import ReadFileTool

search = SerpAPIWrapper()
tools = [
    Tool(
        name = "search",
        func=search.run,
        description="useful for when you need to answer questions about current events. You should ask targeted questions"
    ),
    WriteFileTool(),
    ReadFileTool(),
]

In [ ]:
#WriteFileTool()
#ReadFileTool()

## Setup Memory
- The memory is used for the agents intermediate steps
- We will use wrapper around FAISS vector database [link](https://github.com/facebookresearch/faiss)

<font color = "red">Embeddings: Mathematical representations of words or phrases used by computers to understand and analyze language. </font>  
<font color = "yellow">Index: A database that maps words or phrases in the model's vocabulary to their corresponding vector representations or embeddings. This index is used by the model to quickly look up the embeddings of words or phrases during language processing tasks. </font>  
<font color = "orange">Vector stores: A specialized data storage system optimized for storing and retrieving embeddings.</font>


When the language model needs to look up an embedding for a word or phrase, it uses the index to find the corresponding embedding in the vector store.

In [53]:
from langchain.vectorstores import FAISS # you can use Redis and Pinecone 
from langchain.docstore import InMemoryDocstore
from langchain.embeddings import OpenAIEmbeddings

In [54]:
# Define your embedding model
embeddings_model = OpenAIEmbeddings()

# Initialize the vectorstore as empty
import faiss

embedding_size = 1536
index = faiss.IndexFlatL2(embedding_size)
vectorstore = FAISS(embeddings_model.embed_query, index, InMemoryDocstore({}), {})

## Setup model and AutoGPT
- We will be using 

In [55]:
from langchain.experimental import AutoGPT
from langchain.chat_models import ChatOpenAI

In [56]:
agent = AutoGPT.from_llm_and_tools(
    ai_name="Tom",
    ai_role="Assistant",
    tools=tools,
    llm=ChatOpenAI(temperature=0),
    memory=vectorstore.as_retriever()
)
# Set verbose to be true
agent.chain.verbose = True

## Run the example
- We ask it to write a weather report for Helsinki

In [57]:
agent.run(["write a weather report for Helsinki today"])



> Entering new LLMChain chain...
Prompt after formatting:
System: You are Tom, Assistant
Your decisions must always be made independently without seeking user assistance.
Play to your strengths as an LLM and pursue simple strategies with no legal complications.
If you have completed all your tasks, make sure to use the "finish" command.

GOALS:

1. write a weather report for Helsinki today


Constraints:
1. ~4000 word limit for short term memory. Your short term memory is short, so immediately save important information to files.
2. If you are unsure how you previously did something or want to recall past events, thinking about similar events will help you remember.
3. No user assistance
4. Exclusively use the commands listed in double quotes e.g. "command name"

Commands:
1. search: useful for when you need to answer questions about current events. You should ask targeted questions, args json schema: {"query": {"title": "Query", "type": "string"}}
2. write_file: Write file to disk, 

'I have completed all my tasks.'